In [1]:
import pandas as pd
import json

In [ ]:
df = pd.read_csv("/data/raw_stock_indicator.csv")

In [ ]:
def json_parser(x):
    try:
        return json.loads(x)["symbol"]
    except Exception as e:
        print(e)
        return ""

In [ ]:
df["company_symbol"] = df["info_json"].map(lambda x: json_parser(x))  

In [ ]:
df["company_symbol"].unique()

In [36]:
df.columns

Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA', 'Total Unusual Items',
       'Total Unusual Items Excluding Goodwill',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBIT',
       'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Otherunder Preferred Stock Dividend',
       'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Restructuring And Mergern Acquisition',
       'Net Non Operating Interest Inco

In [37]:
cols = ["Total Revenue", "Operating Revenue", "Gross Profit", "Year", "Month", "company_symbol" ,"company_symbol", "Open", "Volume"]

In [43]:
df1 = df[cols]
df1["year_month"] = df1["Year"]+df1["Month"]
df1

<ipython-input-43-724f27fa80f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["year_month"] = df1["Year"]+df1["Month"]


,Total Revenue,Operating Revenue,Gross Profit,Year,Month,company_symbol,company_symbol,Open,Volume,year_month
0,8.702100e+10,8.702100e+10,3.103400e+10,2023,6,LTIM.NS,LTIM.NS,5084.918182,581634.0,2029
1,8.691000e+10,8.691000e+10,3.110300e+10,2023,3,LTIM.NS,LTIM.NS,4603.711157,328337.0,2026
2,8.620000e+10,8.620000e+10,3.147800e+10,2022,12,LTIM.NS,LTIM.NS,NaN,NaN,2034
3,4.836700e+10,4.836700e+10,1.751800e+10,2022,9,LTIM.NS,LTIM.NS,4345.868216,410491.0,2031
4,4.522800e+10,4.522800e+10,1.650500e+10,2022,6,LTIM.NS,LTIM.NS,4019.437045,374541.0,2028
...,...,...,...,...,...,...,...,...,...,...
999,2.139500e+10,2.139500e+10,1.436700e+10,2022,6,BIOCON.NS,BIOCON.NS,307.303354,2216979.0,2028
1000,1.361570e+08,1.361570e+08,6.936500e+06,2023,3,BIOFILCHEM.NS,BIOFILCHEM.NS,42.500000,27066.0,2026
1001,1.494400e+07,1.494400e+07,4.228000e+06,2022,12,BIOFILCHEM.NS,BIOFILCHEM.NS,NaN,NaN,2034
1002,2.619600e+07,2.619600e+07,3.750000e+06,2022,9,BIOFILCHEM.NS,BIOFILCHEM.NS,47.900002,7374.0,2031


In [39]:
df1['price_change'] = df1.groupby(['Year', 'Month'])['Open'].shift(1)

<ipython-input-39-d2d868969688>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['price_change'] = df1.groupby(['Year', 'Month'])['Open'].shift(1)


In [40]:
df1

,Total Revenue,Operating Revenue,Gross Profit,Year,Month,company_symbol,company_symbol,Open,Volume,price_change
0,8.702100e+10,8.702100e+10,3.103400e+10,2023,6,LTIM.NS,LTIM.NS,5084.918182,581634.0,NaN
1,8.691000e+10,8.691000e+10,3.110300e+10,2023,3,LTIM.NS,LTIM.NS,4603.711157,328337.0,NaN
2,8.620000e+10,8.620000e+10,3.147800e+10,2022,12,LTIM.NS,LTIM.NS,NaN,NaN,NaN
3,4.836700e+10,4.836700e+10,1.751800e+10,2022,9,LTIM.NS,LTIM.NS,4345.868216,410491.0,NaN
4,4.522800e+10,4.522800e+10,1.650500e+10,2022,6,LTIM.NS,LTIM.NS,4019.437045,374541.0,NaN
...,...,...,...,...,...,...,...,...,...,...
999,2.139500e+10,2.139500e+10,1.436700e+10,2022,6,BIOCON.NS,BIOCON.NS,307.303354,2216979.0,110.489410
1000,1.361570e+08,1.361570e+08,6.936500e+06,2023,3,BIOFILCHEM.NS,BIOFILCHEM.NS,42.500000,27066.0,200.256227
1001,1.494400e+07,1.494400e+07,4.228000e+06,2022,12,BIOFILCHEM.NS,BIOFILCHEM.NS,NaN,NaN,NaN
1002,2.619600e+07,2.619600e+07,3.750000e+06,2022,9,BIOFILCHEM.NS,BIOFILCHEM.NS,47.900002,7374.0,283.531562
